# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [133]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import csv
import json
# Import API key
from api_keys import g_key
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [134]:
csv_file = '/Users/bitty/python-api-challenge/python-api-challenge/WeatherPy/cities.csv'
city_data = pd.read_csv(csv_file)
city_data.head()


,Unnamed: 0,city,latitude,longitude,max_temperature,cloudiness,wind_speed,humidity,country,date
0,0,Punta Arenas,-53.1500,-70.9167,42.80,0,3.44,75,CL,1619326413
1,1,Praia,14.9215,-23.5087,69.80,75,12.66,73,CV,1619326413
2,2,Rikitea,-23.1203,-134.9692,78.66,100,12.82,76,PF,1619326414
3,3,Kuytun,54.3419,101.5036,43.41,97,8.70,55,RU,1619326414
4,4,Yining,29.0361,114.5636,61.52,100,3.71,91,CN,1619326414


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [135]:
# Create a Heatmap layer

locations = city_data[["latitude", "longitude"]].astype(float)
weights = city_data["humidity"].astype(float)

fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=weights, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)


fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [136]:
#dropping rows with missing info
city_data.head()
city_data = city_data.dropna(how='any')
city_data

,Unnamed: 0,city,latitude,longitude,max_temperature,cloudiness,wind_speed,humidity,country,date
0,0,Punta Arenas,-53.1500,-70.9167,42.80,0,3.44,75,CL,1619326413
1,1,Praia,14.9215,-23.5087,69.80,75,12.66,73,CV,1619326413
2,2,Rikitea,-23.1203,-134.9692,78.66,100,12.82,76,PF,1619326414
3,3,Kuytun,54.3419,101.5036,43.41,97,8.70,55,RU,1619326414
4,4,Yining,29.0361,114.5636,61.52,100,3.71,91,CN,1619326414
...,...,...,...,...,...,...,...,...,...,...
579,579,Sawtell,-30.3833,153.1000,71.60,3,10.36,64,AU,1619326640
580,580,Portlethen,57.0547,-2.1307,32.00,100,1.14,100,GB,1619326640
581,581,Balkhash,46.8440,74.9804,48.20,0,29.08,34,KZ,1619326641
582,582,Boyolangu,-8.1181,111.8935,85.26,12,6.67,64,ID,1619326641


In [137]:
#using loc to pull rows with ideal weather conditions
#conditions:
    #A max temperature lower than 80 degrees but higher than 70.
    #Wind speed less than 10 mph.
    #Zero cloudiness.
ideal_weather = city_data.loc[city_data['max_temperature'] > 70, :] 
ideal_weather = ideal_weather.loc[ideal_weather['max_temperature'] < 80, :]
ideal_weather = ideal_weather.loc[ideal_weather['wind_speed'] < 10, :]
ideal_weather = ideal_weather.loc[ideal_weather['cloudiness'] == 0, :]
ideal_weather

,Unnamed: 0,city,latitude,longitude,max_temperature,cloudiness,wind_speed,humidity,country,date
197,197,Caravelas,-17.7125,-39.2481,73.00,0,4.76,90,BR,1619326490
323,323,Pitimbu,-7.4706,-34.8086,73.40,0,4.61,100,BR,1619326540
362,362,Sakaraha,-22.9000,44.5333,71.33,0,3.89,67,MG,1619326556
448,448,Prado,-17.3411,-39.2208,73.60,0,5.95,89,BR,1619326588


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [138]:
#create a dataframe for only ideal weather locations, add hotel name column
hotel_df = pd.DataFrame(ideal_weather)
hotel_df['hotel name'] = ''
hotel_df

,Unnamed: 0,city,latitude,longitude,max_temperature,cloudiness,wind_speed,humidity,country,date,hotel name
197,197,Caravelas,-17.7125,-39.2481,73.00,0,4.76,90,BR,1619326490,
323,323,Pitimbu,-7.4706,-34.8086,73.40,0,4.61,100,BR,1619326540,
362,362,Sakaraha,-22.9000,44.5333,71.33,0,3.89,67,MG,1619326556,
448,448,Prado,-17.3411,-39.2208,73.60,0,5.95,89,BR,1619326588,


In [139]:
# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

locations = "-17.7125, -39.2481"
target_search = "Hotels"
target_radius = 5000
hotel_names = []
# create parameters dictionary to use
params = {
    "location": locations,
    "keyword": target_search,
    "radius": target_radius,
    "key": g_key
}

# run a request using our params dictionary
response = requests.get(base_url, params=params).json()

# Print the json (pretty printed)
print(json.dumps(response, indent=4, sort_keys=True))
name = response['results'][0]['name']
hotel_df.iloc[0,10] = name
hotel_names.append(name)
print(name)

{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -17.7431944,
                    "lng": -39.24093329999999
                },
                "viewport": {
                    "northeast": {
                        "lat": -17.74181127010728,
                        "lng": -39.23957242010727
                    },
                    "southwest": {
                        "lat": -17.74451092989273,
                        "lng": -39.24227207989272
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/generic_business-71.png",
            "name": "Mar Doce de Abrolhos",
            "opening_hours": {
                "open_now": false
            },
            "photos": [
                {
                    "height": 2012,
                    "html_attributions": [
   

In [140]:
hotel_df

,Unnamed: 0,city,latitude,longitude,max_temperature,cloudiness,wind_speed,humidity,country,date,hotel name
197,197,Caravelas,-17.7125,-39.2481,73.00,0,4.76,90,BR,1619326490,Mar Doce de Abrolhos
323,323,Pitimbu,-7.4706,-34.8086,73.40,0,4.61,100,BR,1619326540,
362,362,Sakaraha,-22.9000,44.5333,71.33,0,3.89,67,MG,1619326556,
448,448,Prado,-17.3411,-39.2208,73.60,0,5.95,89,BR,1619326588,


In [141]:
#second location 

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

locations = "-7.4706, -34.8086" 
target_search = "Hotels"
target_radius = 5000
hotel_names = []
# create parameters dictionary to use
params = {
    "location": locations,
    "keyword": target_search,
    "radius": target_radius,
    "key": g_key
}

# run a request using our params dictionary
response = requests.get(base_url, params=params).json()

# Print the json (pretty printed)
print(json.dumps(response, indent=4, sort_keys=True))
name = response['results'][0]['name']
hotel_df.iloc[1,10] = name
hotel_names.append(name)
print(name)

{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -7.444311599999999,
                    "lng": -34.8123435
                },
                "viewport": {
                    "northeast": {
                        "lat": -7.443322170107279,
                        "lng": -34.81125292010727
                    },
                    "southwest": {
                        "lat": -7.446021829892723,
                        "lng": -34.81395257989272
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png",
            "name": "Asenza Beach Resort",
            "opening_hours": {
                "open_now": true
            },
            "photos": [
                {
                    "height": 6936,
                    "html_attributions": [
              

In [142]:
#third location 

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

locations = "-22.9000, 44.5333" 
target_search = "Hotels"
target_radius = 5000
hotel_names = []
# create parameters dictionary to use
params = {
    "location": locations,
    "keyword": target_search,
    "radius": target_radius,
    "key": g_key
}

# run a request using our params dictionary
response = requests.get(base_url, params=params).json()

# Print the json (pretty printed)
print(json.dumps(response, indent=4, sort_keys=True))
name = response['results'][0]['name']
hotel_df.iloc[2,10] = name
hotel_names.append(name)
print(name)

{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -22.9127983,
                    "lng": 44.53882
                },
                "viewport": {
                    "northeast": {
                        "lat": -22.91140727010728,
                        "lng": 44.54018077989272
                    },
                    "southwest": {
                        "lat": -22.91410692989272,
                        "lng": 44.53748112010728
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png",
            "name": "H\u00f4tel 2000",
            "photos": [
                {
                    "height": 1445,
                    "html_attributions": [
                        "<a href=\"https://maps.google.com/maps/contrib/108753295304202529473\">Laitiko Yrdna

In [143]:
#fourth location 

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

locations = "-17.3411, -39.2208" 
target_search = "Hotels"
target_radius = 5000
hotel_names = []
# create parameters dictionary to use
params = {
    "location": locations,
    "keyword": target_search,
    "radius": target_radius,
    "key": g_key
}

# run a request using our params dictionary
response = requests.get(base_url, params=params).json()

# Print the json (pretty printed)
print(json.dumps(response, indent=4, sort_keys=True))
name = response['results'][0]['name']
hotel_df.iloc[3,10] = name
hotel_names.append(name)
print(name)

osEXruiHZY2JL2Nne6go5TzAEGWBSDkZ-ZVisam2r6WBMD-JzvUqXHNBcTXn2LS_Y6TGUNDuUc4xa99CvixM6ofxBR38H15tUlQqsSeQ8pNd-fDsUpc1BXsY1_uoOhr_Wla4IyyJbR-5Csu0oDn3s8N39m_VAu2D-9hNuNZ8a1Gq9",
                    "width": 1280
                }
            ],
            "place_id": "ChIJ2XKlBOH7NAcRVj5xPuMkEv8",
            "plus_code": {
                "compound_code": "MQ4J+64 Prado, State of Bahia, Brazil",
                "global_code": "59J2MQ4J+64"
            },
            "rating": 4.6,
            "reference": "ChIJ2XKlBOH7NAcRVj5xPuMkEv8",
            "scope": "GOOGLE",
            "types": [
                "lodging",
                "point_of_interest",
                "establishment"
            ],
            "user_ratings_total": 196,
            "vicinity": "Rua Alfredo Horcades, 73 - Centro, Prado"
        },
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -17.3287462,
                    "lng

In [144]:
hotel_df

,Unnamed: 0,city,latitude,longitude,max_temperature,cloudiness,wind_speed,humidity,country,date,hotel name
197,197,Caravelas,-17.7125,-39.2481,73.00,0,4.76,90,BR,1619326490,Mar Doce de Abrolhos
323,323,Pitimbu,-7.4706,-34.8086,73.40,0,4.61,100,BR,1619326540,Asenza Beach Resort
362,362,Sakaraha,-22.9000,44.5333,71.33,0,3.89,67,MG,1619326556,Hôtel 2000
448,448,Prado,-17.3411,-39.2208,73.60,0,5.95,89,BR,1619326588,Cahy Praia Hotel


In [145]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{hotel name}</dd>
<dt>City</dt><dd>{city}</dd>
<dt>Country</dt><dd>{country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["latitude", "longitude"]]

In [146]:
print(hotel_info)

['\n<dl>\n<dt>Name</dt><dd>Mar Doce de Abrolhos</dd>\n<dt>City</dt><dd>Caravelas</dd>\n<dt>Country</dt><dd>BR</dd>\n</dl>\n', '\n<dl>\n<dt>Name</dt><dd>Asenza Beach Resort</dd>\n<dt>City</dt><dd>Pitimbu</dd>\n<dt>Country</dt><dd>BR</dd>\n</dl>\n', '\n<dl>\n<dt>Name</dt><dd>Hôtel 2000</dd>\n<dt>City</dt><dd>Sakaraha</dd>\n<dt>Country</dt><dd>MG</dd>\n</dl>\n', '\n<dl>\n<dt>Name</dt><dd>Cahy Praia Hotel</dd>\n<dt>City</dt><dd>Prado</dd>\n<dt>Country</dt><dd>BR</dd>\n</dl>\n']


In [151]:
hotel_df

,Unnamed: 0,city,latitude,longitude,max_temperature,cloudiness,wind_speed,humidity,country,date,hotel name
197,197,Caravelas,-17.7125,-39.2481,73.00,0,4.76,90,BR,1619326490,Mar Doce de Abrolhos
323,323,Pitimbu,-7.4706,-34.8086,73.40,0,4.61,100,BR,1619326540,Asenza Beach Resort
362,362,Sakaraha,-22.9000,44.5333,71.33,0,3.89,67,MG,1619326556,Hôtel 2000
448,448,Prado,-17.3411,-39.2208,73.60,0,5.95,89,BR,1619326588,Cahy Praia Hotel


In [153]:
#tried to write out the info box code myself, needs troubleshooting
info_box_data = [
    {'name': 'Mar Doce de Abrolhos', 'location': (-17.7125,-39.2481), 'city': 'Caravelas', 'country':'BR' },
    {'name': 'Asenza Beach Resort', 'location': (-7.4706,-34.8086), 'city': 'Pitimbu', 'country':'BR' },
    {'name': 'Hôtel 2000', 'location': (-22.9000,44.5333), 'city': 'Sakaraha', 'country':'MG' },
    {'name': 'Cahy Praia Hotel', 'location': (-17.3411,-39.2208), 'city': 'Prado', 'country':'BR' }
]

In [155]:
info_box_template = """
<dl>
<dt>Name</dt><dd>{name}</dd>
<dt>City</dt><dd>{city}</dd>
<dt>Country</dt><dd>{country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name

# Add marker layer ontop of heat map
hotel_try = [info_box_template.format(**data) for data in info_box_data]
marker_layer = gmaps.marker_layer(info_box_data, info_box_content=hotel_try)
fig.add_layer(marker_layer)

ValueError: too many values to unpack (expected 2)

In [ ]:

locations = hotel_df[["latitude", "longitude"]]
weights = hotel_df["humidity"].astype(float)

fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=weights, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)


fig.add_layer(heat_layer)



fig
# Display figure